<span style="font-size: 2em; font-weight:bold">AI 70's Country</span>

In [11]:
import scipy
import numpy as np
import matplotlib
import pandas as pd
import statsmodels
import sklearn
import tensorflow
from keras.models import Sequential, load_model
from keras.layers import Dense,LSTM
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
import string
import pyarrow as pa


It was much easier to copy lyrics into a spreadsheet, save as csv, and then convert to straight text than it was to copy directly into a text document.

In [2]:
lyricsCSV = pd.read_csv('lyrics.csv',encoding='ISO-8859-1')
lyricsCSV.to_csv('lyrics.txt',sep='\t',index=False)
l = open('lyrics.txt','r')
lyrics = l.read()
l.close()

Remove lines and print

In [3]:
tokens = lyrics.split()
lyrics = ' '.join(tokens)
print(lyrics)

I beg your pardon I never promised you a rose garden Along with the sunshine There's gotta be a little rain some time When you take you gotta give so live and let live Or let go oh-whoa-whoa-whoa I beg your pardon I never promised you a rose garden I could promise you things like big diamond rings But you don't find roses growin' on stalks of clover So you better think it over Well if sweet-talkin' you could make it come true I would give you the world right now on a silver platter But what would it matter So smile for a while and let's be jolly Love shouldn't be so melancholy Come along and share the good times while we can I beg your pardon I never promised you a rose garden Along with the sunshine There's gotta be a little rain some time I beg your pardon I never promised you a rose garden I could sing you a tune and promise you the moon But if that's what it takes to hold you I'd just as soon let you go But there's one thing I want you to know You better look before you leap, still

We will build sequences of characters that will then be used to predict a final character

In [4]:
length = 100 # Length of the characer sequences (because we have so much verbage,
             # we can use a relatively large number)
sequences = list()
for i in range(length, len(lyrics)):
    seq = lyrics[i-length:i+1]
    sequences.append(seq)

Create and save a .txt file of our sequences with line endings

In [5]:
data = '\n'.join(sequences)
file = open('char_sequences.txt','w')
file.write(data)
file.close()

Create a dictionary of character:number mappings 

In [6]:
file = open('char_sequences.txt','r')
raw_text = file.read()
file.close()

lines = raw_text.split('\n')

chars = sorted(list(set(raw_text)))
mapping = dict((c, i) for i, c in enumerate(chars))

Use the dictionary to create sequences of numbers only (numbers that describe the characters)

In [7]:
sequences = list()
for line in lines:
    encoded_seq = [mapping[char] for char in line]
    sequences.append(encoded_seq)
    
print(sequences)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Create input sets (with 99 characters) and output sets (1 character) and then one-hot code the sets so we can use them to train the model.

In [9]:
vocab_size = len(mapping)
sequences = np.array(sequences)
X, y = sequences[:,:-1], sequences[:,-1]
sequences = [to_categorical(x, num_classes=vocab_size) for x in X] #one-hot code input
X = np.array(sequences)
y = to_categorical(y, num_classes=vocab_size) #one-hot code output

Fit the model  (<span style="color: red">this may take some time</span>)

In [18]:
units = 512
epochs = 100



# define and fit model
model = Sequential()
model.add(LSTM(units, input_shape=(X.shape[1], X.shape[2]))) # changed verbosity to 1  This takes a long time to run
                                                             # so you will want verbosity at some level.
model.add(Dense(vocab_size, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=epochs, verbose=1)
model.save('model.units512')

Epoch 1/100
29904/29904 [==============================] - 378s 13ms/step - loss: 2.6382 - acc: 0.2929
Epoch 2/100
29904/29904 [==============================] - 378s 13ms/step - loss: 2.0663 - acc: 0.4072
Epoch 3/100
29904/29904 [==============================] - 375s 13ms/step - loss: 1.7803 - acc: 0.4848
Epoch 4/100
29904/29904 [==============================] - 371s 12ms/step - loss: 1.5133 - acc: 0.5593
Epoch 5/100
29904/29904 [==============================] - 378s 13ms/step - loss: 1.2440 - acc: 0.6419
Epoch 6/100
29904/29904 [==============================] - 443s 15ms/step - loss: 0.9956 - acc: 0.7102
Epoch 7/100
29904/29904 [==============================] - 466s 16ms/step - loss: 0.7891 - acc: 0.7715
Epoch 8/100
29904/29904 [==============================] - 470s 16ms/step - loss: 0.6106 - acc: 0.8230
Epoch 9/100
29904/29904 [==============================] - 416s 14ms/step - loss: 0.4684 - acc: 0.8655
Epoch 10/100
29904/29904 [==============================] - 382s 13ms/ste

29904/29904 [==============================] - 1143s 38ms/step - loss: 0.0445 - acc: 0.9872
Epoch 80/100
29904/29904 [==============================] - 1005s 34ms/step - loss: 0.0309 - acc: 0.9920
Epoch 81/100
29904/29904 [==============================] - 1076s 36ms/step - loss: 0.0305 - acc: 0.9926
Epoch 82/100
29904/29904 [==============================] - 1065s 36ms/step - loss: 0.0670 - acc: 0.9819
Epoch 83/100
29904/29904 [==============================] - 978s 33ms/step - loss: 0.0536 - acc: 0.9845
Epoch 84/100
29904/29904 [==============================] - 987s 33ms/step - loss: 0.0346 - acc: 0.9903
Epoch 85/100
29904/29904 [==============================] - 1041s 35ms/step - loss: 0.0256 - acc: 0.9932
Epoch 86/100
29904/29904 [==============================] - 1033s 35ms/step - loss: 0.0314 - acc: 0.9912
Epoch 87/100
29904/29904 [==============================] - 938s 31ms/step - loss: 0.0653 - acc: 0.9816
Epoch 88/100
29904/29904 [==============================] - 1088s 36ms/

In [18]:
model.save('model.h5')

In [12]:
model = load_model('model.h5')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [20]:
model = load_model('model.units512')

Create a function that encodes a kickoff text string and then plugs it into our trained model

In [13]:
# generate a sequence of characters with a language model
def generate_seq(model, mapping, seq_length, seed_lyric, n_chars):
    lyrics = seed_lyric
    for __ in range(n_chars):
    # encode the characters as integers
        encoded = [mapping[char] for char in lyrics]
    # truncate sequences to a fixed length
        encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
    # one hot encode
        encoded = to_categorical(encoded, num_classes=len(mapping))
    # predict character
        yhat = model.predict_classes(encoded, verbose=0)
    # reverse map integer to character
        out_char = ''
        for char, index in mapping.items():
            if index == yhat:
                out_char = char
                break
    # append to input
        lyrics += char
    return lyrics

Is there a song other than Stairway to Heaven that I could have used for the kickoff sequence?

In [14]:
startLyrics = "I beg your pardon I never promised you a rose garden Along with the sunshine There's gotta be a litt"

Run the model and print the lyrics

In [15]:
lyricsFinal = generate_seq(model,mapping,length,startLyrics,1000)
print(lyricsFinal)

I beg your pardon I never promised you a rose garden Along with the sunshine There's gotta be a little rain some time I'm along and someone they promised you can I'd smile with you Jolene, Jolene Please don't take my hanestong cowboy Well than the fightin' better that I held wath the froer every the good to she day you ther whe way for by ckind you harm you gotta forget I've fich And I can't sun your know where to tase you the son't semeted the barroom lett the way of livin' over the shonin' me alrobout he was ond me And I really day I'd talla wall alwnis and that home and they promise you the bary, meos, ten I wiln if he was a-meeting you here today To take you to his mansion in the sky? Delta Dawn, what's that flower you have on? Could it be a faded rose from days gone by? And did I hear you say he was a-meeting you here today To take you the soul alone And there got I've a good-tety some talking of the floosed my the good times whith a all good my son I said ""was all door live mer 

neurons = 75 
epochs = 100
vocab_size = len(mapping)


# define and fit model
model2 = Sequential()
model2.add(LSTM(neurons, input_shape=(X.shape[1], X.shape[2])))
model2.add(Dense(vocab_size, activation='softmax'))
model2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model2.fit(X, y, epochs=epochs, verbose=2)

def generate_seq2(model, mapping, seq_length, seed_lyric, n_chars):
    lyrics = seed_lyric
    for __ in range(n_chars):
    # encode the characters as integers
        encoded = [mapping[char] for char in lyrics]
    # truncate sequences to a fixed length
        encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
    # one hot encode
        encoded = to_categorical(encoded, num_classes=len(mapping))
    # predict character
        yhat = model.predict_classes(encoded, verbose=0)
    # reverse map integer to character
        out_char = ''
        for char, index in mapping.items():
            if index == yhat:
                out_char = char
                break
    # append to input
        lyrics += char
    return lyrics

print(generate_seq(model2,mapping,length,startLyrics,1000))